In [ ]:
import torch

# Modificar tensores

1. *Vista (`torch.Tensor.view`). Nos permite cambiar la forma del tensor, similar a lo que hace el método `torch.Tensor.reshape()`, pero con dos grandes diferencias:*
    - *No hace una copia del tensor en memoria. El nuevo tensor referencia al mismo objeto en memoria.*
    - *Solo funciona si el tensor es **contiguo** en memoria.*
        - *Que un tensor sea contiguo quiere decir que sus elementos están guardados de forma secuencial en un bloque de memoria ininterrumpido (vector uni-dimensional).*
    - *Hay operaciones que pueden hacer que un tensor no sea contiguo, por ejemplo, la transposición de un tensor. En este caso, este método nos devolverá un error (se puede llamar al método `torch.Tensor.contiguous()` antes de llamar a `torch.Tensor.view()`).*
2. *Reshape (`torch.Tensor.reshape()`). Nos permite cambiar la forma del tensor sin cambiar sus elementos. Por ejemplo, cambia la dimensión de $3\times 2$ a $2\times 3$.*
    - *No requiere que el tensor sea contiguo en memoria. Si el tensor es contiguo, llama al método `torch.Tensor.view()`. Si no es contiguo, entonces primero llama al método `torch.Tensor.contiguous()`, creando una copia del tensor en memoria que sí es contigua.*
    - *Si no estamos seguros de si el tensor es contiguo o no, es mejor usar `torch.Tensor.reshape()` en lugar de `torch.Tensor.view()`.*
3. *Concatenar. Hay varias opciones disponibles para concatenar tensores:*
    -  *`torch.cat()`. Permite concatenar tensores sobre una dimensión existente, resultando en un tensor con la misma cantidad de dimensiones. Por ejemplo, si tenemos dos tensor de $2\times 2$, y los concatenamos usando `dim=0` (en este caso filas), el resultado será un tensor de $4\times 2$, mientras que si los concatenamos usando `dim=1` (en este caso columnas), el resultado será un tensor de $2\times 4$.*
        - *No es necesario que la dimensión sobre la que concatenamos tenga el mismo tamaño, pero las dimensiones restantes deben ser iguales.*
    - *`torch.stack()`. Permite concatenar tensores sobre una nueva dimensión, resultando en un tensor con una dimensión adicional. Por ejemplo, si tenemos dos tensor de $2\times 2$, y los concatenamos usando `dim=0`, entonces nos devuelve un tensor de dimensión $2\times2\times2$.*
        - *Ambos tensores tienen que tener el mismo tamaño.*
4. *Eliminar/agregar dimensions. La función `torch.squeezed()` nos permite eliminar dimensiones de tamaño 1, mientras que la función `torch.unsqueezed()` nos permite agregar dimensiones de tamaño 1.*
5. *Permutar dimensiones. Con la función `torch.permuted()` podemos re-acomodar las dimensiones de un tensor.*

## Vistas

In [47]:
X = torch.arange(0, 10)
X = X.type(torch.float32)
X

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [48]:
y = X.view(2, 5)
y

tensor([[0., 1., 2., 3., 4.],
        [5., 6., 7., 8., 9.]])

*Creamos un nuevo tensor `y`, que es una vista del tensor `X`, de tamaño $2\times 5$. Ahora, cambiar uno de los elementos del tensor `y` afectaría al tensor `X`? Sí, porque `y` hace referencia al mismo objeto en memoria que el tensor `X`, por lo tanto, si cambiamos `y` también vamos a cambiar a `X`.*

In [49]:
y[:, 0] = torch.tensor([10, 20], dtype=torch.float32)
y

tensor([[10.,  1.,  2.,  3.,  4.],
        [20.,  6.,  7.,  8.,  9.]])

In [50]:
X

tensor([10.,  1.,  2.,  3.,  4., 20.,  6.,  7.,  8.,  9.])

## Reshape

In [68]:
X = torch.rand(size=(2, 3))
X

tensor([[0.3700, 0.5732, 0.2554],
        [0.2901, 0.9397, 0.7976]])

In [69]:
y = X.reshape(3, -1) # -1 means that the size of that dimension is defined automatically
y

tensor([[0.3700, 0.5732],
        [0.2554, 0.2901],
        [0.9397, 0.7976]])

*Podemos ver que en este caso el método `reshape` nos devuelve una vista del tensor `X` porque un cambio en `y` impacta en el tensor `X`. Esto porque el tensor `X` sigue siendo contiguo en memoria.*

In [70]:
y[0, :] = torch.tensor([1, 1])
y

tensor([[1.0000, 1.0000],
        [0.2554, 0.2901],
        [0.9397, 0.7976]])

In [71]:
X

tensor([[1.0000, 1.0000, 0.2554],
        [0.2901, 0.9397, 0.7976]])

*Creamos un nuevo tensor, `z`, que es la versión transpuesta de `X`, y a partir de ese nuevo tensor creamos el tensor `y`. Ahora, si realizamos un cambio en `y`, este cambio no va a impactar en `X`. Por qué ocurre esto? Porque al crear el tensor `z` a partir de la versión transpuesta de `X`, ese tensor deja de ser contiguo en memoria, y entonces el método `reshape()` crea una copia de `z` y no una vista.*

In [72]:
z = X.T
z

tensor([[1.0000, 0.2901],
        [1.0000, 0.9397],
        [0.2554, 0.7976]])

In [73]:
y = z.reshape(2, -1)
y

tensor([[1.0000, 0.2901, 1.0000],
        [0.9397, 0.2554, 0.7976]])

In [74]:
y[:, 0] = torch.tensor([0, 0])
y

tensor([[0.0000, 0.2901, 1.0000],
        [0.0000, 0.2554, 0.7976]])

In [75]:
X

tensor([[1.0000, 1.0000, 0.2554],
        [0.2901, 0.9397, 0.7976]])

## Concatenar

In [77]:
tensor_A = torch.rand(size=(2, 3))
tensor_B = torch.rand(size=(2, 3))

print(f'tensor_A:\n {tensor_A}')
print(f'tensor_B:\n {tensor_B}')

tensor_A:
 tensor([[0.9063, 0.8582, 0.9965],
        [0.8017, 0.4278, 0.9973]])
tensor_B:
 tensor([[0.9701, 0.1752, 0.0940],
        [0.1659, 0.8378, 0.3756]])


*Si usamos `torch.Tensor.cat()`, podemos concatenar dos tensores sobre una dimensión existente (mantiene el tamaño de las dimensiones sobre las que no concatenamos). Por ejemplo, si concatenamos sobre la `dim=0`, nos queda un tensor de $4\times 2$ (se mantiene el tamaño de la segunda dimensión).*

In [80]:
tensor_C = torch.cat((tensor_A, tensor_B), dim=0)
print(f'Original shape: tensor_A = {tensor_A.shape} and tensor_B = {tensor_B.shape}')
print(f'Shape after concatenation: tensor_C = {tensor_C.shape}\n')
print(f'New tensor:\n {tensor_C}')

Original shape: tensor_A = torch.Size([2, 3]) and tensor_B = torch.Size([2, 3])
Shape after concatenation: tensor_C = torch.Size([4, 3])

New tensor:
 tensor([[0.9063, 0.8582, 0.9965],
        [0.8017, 0.4278, 0.9973],
        [0.9701, 0.1752, 0.0940],
        [0.1659, 0.8378, 0.3756]])


*Si, por otro lado, usamos la dimensión `dim=1`, nos queda un tensor de $2\times6$.*

In [81]:
tensor_C = torch.cat((tensor_A, tensor_B), dim=1)
print(f'Original shape: tensor_A = {tensor_A.shape} and tensor_B = {tensor_B.shape}')
print(f'Shape after concatenation: tensor_C = {tensor_C.shape}\n')
print(f'New tensor:\n {tensor_C}')

Original shape: tensor_A = torch.Size([2, 3]) and tensor_B = torch.Size([2, 3])
Shape after concatenation: tensor_C = torch.Size([2, 6])

New tensor:
 tensor([[0.9063, 0.8582, 0.9965, 0.9701, 0.1752, 0.0940],
        [0.8017, 0.4278, 0.9973, 0.1659, 0.8378, 0.3756]])


*Si usamos `torch.Tensor.stack()`, creamos una nueva dimensión. Entonces, en este caso nos queda un tensor de tres dimensiones a partir de concatenar dos tensores de dos dimensiones. En este caso es necesario que ambos tensores tengan las mismas dimensiones.*

In [82]:
tensor_C = torch.stack((tensor_A, tensor_B), dim=0)
print(f'Original shape: tensor_A = {tensor_A.shape} and tensor_B = {tensor_B.shape}')
print(f'Shape after concatenation: tensor_C = {tensor_C.shape}\n')
print(f'New tensor:\n {tensor_C}')

Original shape: tensor_A = torch.Size([2, 3]) and tensor_B = torch.Size([2, 3])
Shape after concatenation: tensor_C = torch.Size([2, 2, 3])

New tensor:
 tensor([[[0.9063, 0.8582, 0.9965],
         [0.8017, 0.4278, 0.9973]],

        [[0.9701, 0.1752, 0.0940],
         [0.1659, 0.8378, 0.3756]]])


## Eliminar/agregar dimensiones

*La función `torch.squeezed()` nos permite eliminar dimensiones de tamaño 1 de un tensor. El argumento `dim` nos permite elegir que dimensiones queremos eliminar.*
- *Hay que tener en cuenta que el tensor resultante de esta función es una vista (`torch.Tensor.view`), lo que quiere decir que comparte el mismo objeto en memoria que el tensor original.*

In [93]:
X = torch.rand(size=(1, 2, 3))
X

tensor([[[0.1866, 0.0648, 0.6463],
         [0.6808, 0.5452, 0.9616]]])

*Quitamos la primer dimensión que tiene tamaño 1. El tensor resultante, `tensor_X_squeezed`, es una matriz de $2\times 3$.*

In [87]:
tensor_X_squeezed = torch.squeeze(X)
print(f'Original shape: {X.shape}')
print(f'Shape after squeezing: {tensor_X_squeezed.shape}\n')
print(f'New tensor:\n {tensor_X_squeezed}')

Original shape: torch.Size([1, 2, 3])
Shape after squeezing: torch.Size([2, 3])

New tensor:
 tensor([[0.9786, 0.7882, 0.6897],
        [0.1621, 0.8248, 0.8890]])


*Por otro lado, la función `torch.unsqueezed()` nos permite agregar una dimensión (debemos especificarla). En el código debajo volvemos a agregar la dimensión que quitamos anteriormente.*

In [92]:
tensor_X_unsqueezed = torch.unsqueeze(tensor_X_squeezed, dim=0)
print(f'Shape squeezed tensor: {tensor_X_squeezed.shape}')
print(f'Shape after unsqueezing: {tensor_X_unsqueezed.shape}\n')
print(f'New tensor:\n {tensor_X_unsqueezed}')

Shape squeezed tensor: torch.Size([2, 3])
Shape after unsqueezing: torch.Size([2, 1, 3])

New tensor:
 tensor([[[0.9786, 0.7882, 0.6897]],

        [[0.1621, 0.8248, 0.8890]]])


## Permutar dimensiones

*La función `torch.permuted()` nos permite re-acomodar las dimensiones de un tensor. Por ejemplo, tenemos un tensor de $1024\times1024\times3$ (podemos pensar que es una imagen, donde las dos primeras dimensiones hacen referencia a la altura y el largo, y la última dimensión hace referencia a los canales de colores -RGB-). Podríamos cambiar la última dimensión para que sea la primera (i.e., `dim=0`).*

In [94]:
image = torch.rand(size=(1024, 1024, 3))
image

tensor([[[0.9683, 0.9730, 0.2627],
         [0.8097, 0.8765, 0.0550],
         [0.6498, 0.4069, 0.7136],
         ...,
         [0.3865, 0.8466, 0.9223],
         [0.7619, 0.6037, 0.4737],
         [0.6288, 0.7732, 0.5016]],

        [[0.7348, 0.9181, 0.7637],
         [0.3168, 0.6106, 0.4738],
         [0.4297, 0.4724, 0.0670],
         ...,
         [0.6849, 0.3207, 0.5561],
         [0.5420, 0.3546, 0.4909],
         [0.5327, 0.2005, 0.7087]],

        [[0.5963, 0.4694, 0.3344],
         [0.8822, 0.2125, 0.4717],
         [0.0236, 0.4680, 0.9261],
         ...,
         [0.5447, 0.1777, 0.9908],
         [0.6046, 0.7683, 0.6753],
         [0.4742, 0.0015, 0.0772]],

        ...,

        [[0.8567, 0.7679, 0.7789],
         [0.4698, 0.0566, 0.6014],
         [0.8258, 0.4664, 0.5827],
         ...,
         [0.7187, 0.1043, 0.7711],
         [0.3493, 0.7985, 0.1038],
         [0.0346, 0.3198, 0.4269]],

        [[0.6278, 0.5091, 0.4229],
         [0.0168, 0.8149, 0.7544],
         [0.

*Para permutar las dimensiones del tensor, tenemos que especificar las dimensiones del tensor actual que queremos en la dimensión del nuevo tensor. En este caso, queremos que el nuevo orden sea: la tercera dimensión primero (i.e., `dim=2`), después la primer dimensión (i.e., `dim=0`) y finalmente la segunda dimensión (i.e., `dim=1`).*

In [95]:
image_permuted = torch.permute(image, dims=(2, 0, 1))
print(f'Original shape: {image.shape}')
print(f'Shape after permuting: {image_permuted.shape}\n')
print(f'New tensor:\n {image_permuted}')

Original shape: torch.Size([1024, 1024, 3])
Shape after permuting: torch.Size([3, 1024, 1024])

New tensor:
 tensor([[[0.9683, 0.8097, 0.6498,  ..., 0.3865, 0.7619, 0.6288],
         [0.7348, 0.3168, 0.4297,  ..., 0.6849, 0.5420, 0.5327],
         [0.5963, 0.8822, 0.0236,  ..., 0.5447, 0.6046, 0.4742],
         ...,
         [0.8567, 0.4698, 0.8258,  ..., 0.7187, 0.3493, 0.0346],
         [0.6278, 0.0168, 0.8417,  ..., 0.7759, 0.0819, 0.1806],
         [0.7465, 0.6822, 0.9011,  ..., 0.5210, 0.7169, 0.2203]],

        [[0.9730, 0.8765, 0.4069,  ..., 0.8466, 0.6037, 0.7732],
         [0.9181, 0.6106, 0.4724,  ..., 0.3207, 0.3546, 0.2005],
         [0.4694, 0.2125, 0.4680,  ..., 0.1777, 0.7683, 0.0015],
         ...,
         [0.7679, 0.0566, 0.4664,  ..., 0.1043, 0.7985, 0.3198],
         [0.5091, 0.8149, 0.5538,  ..., 0.1778, 0.9297, 0.2597],
         [0.3158, 0.2986, 0.9336,  ..., 0.7761, 0.6780, 0.3433]],

        [[0.2627, 0.0550, 0.7136,  ..., 0.9223, 0.4737, 0.5016],
         [0.76